In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
from torchvision.models import resnet18
import pandas as pd
import numpy as np
from PIL import Image
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
# Load your data
df_train = pd.read_csv("/content/train_with_images.csv")
df_test = pd.read_csv("/content/test_with_images.csv")

print(df_train.shape, df_test.shape)

(16209, 22) (5404, 21)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

# Load your processed data (with image_path)
df_train = pd.read_csv("/content/train_with_images.csv")
df_test = pd.read_csv("/content/test_with_images.csv")

print(f"Train: {df_train.shape}, Test: {df_test.shape}")
print("Columns:", df_train.columns.tolist())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Train: (16209, 22), Test: (5404, 21)
Columns: ['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode', 'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sale_year', 'sale_month', 'image_path']


In [3]:
# ← CRITICAL: REPLACE WITH YOUR EXACT PATH FROM FILES PANEL
IMAGE_BASE_PATH = "/content/drive/MyDrive/images"

# Create image_path column
df_train["image_path"] = IMAGE_BASE_PATH + "/" + df_train.index.astype(str) + ".png"
df_test["image_path"] = IMAGE_BASE_PATH + "/" + df_test.index.astype(str) + ".png"

# Test 3 images exist
for i in [0, 1, 100]:
    path = df_train.iloc[i]["image_path"]
    status = "OK" if os.path.exists(path) else "MISSING"
    print(f"Row {i}: {status}")


Row 0: OK
Row 1: OK
Row 100: OK


In [4]:
# Filter only rows with existing images
valid_mask = [os.path.exists(p) for p in df_train['image_path']]
df_train_valid = df_train[valid_mask].reset_index(drop=True)

tabular_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
                   'waterfront', 'view', 'condition', 'grade', 'sqft_above',
                   'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
                   'lat', 'long', 'sqft_living15', 'sqft_lot15', 'sale_year', 'sale_month']

X_tab = df_train_valid[tabular_features].values
y = df_train_valid['price'].values

scaler = StandardScaler()
X_tab_scaled = scaler.fit_transform(X_tab)

print(f"Valid images: {len(df_train_valid)}/{len(df_train)}")
print("Tabular shape:", X_tab_scaled.shape)


Valid images: 16209/16209
Tabular shape: (16209, 20)


In [5]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

class HouseDataset(Dataset):
    def __init__(self, df, tabular_data, transform=None):
        self.df = df
        self.tabular = tabular_data
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = self.df.iloc[idx]['image_path']
        image = Image.open(img_path).convert('RGB')
        if self.transform:
            image = self.transform(image)
        tabular = torch.FloatTensor(self.tabular[idx])
        price = torch.FloatTensor([self.df.iloc[idx]['price']])
        return image, tabular, price

dataset = HouseDataset(df_train_valid, X_tab_scaled, transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
print("Dataset ready!")


Dataset ready!


In [6]:
class MultimodalModel(nn.Module):
    def __init__(self, tabular_size=20):
        super().__init__()
        self.cnn = resnet18(weights='IMAGENET1K_V1')
        self.cnn.fc = nn.Identity()

        self.tabular_net = nn.Sequential(
            nn.Linear(tabular_size, 128), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(128, 64)
        )

        self.fusion = nn.Sequential(
            nn.Linear(512 + 64, 256), nn.ReLU(), nn.Dropout(0.3),
            nn.Linear(256, 128), nn.ReLU(),
            nn.Linear(128, 1)
        )

    def forward(self, image, tabular):
        img_feat = self.cnn(image)
        tab_feat = self.tabular_net(tabular)
        combined = torch.cat([img_feat, tab_feat], dim=1)
        return self.fusion(combined)

model = MultimodalModel(tabular_size=len(tabular_features))
print(model)


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:00<00:00, 149MB/s]

MultimodalModel(
  (cnn): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

In [7]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Allow slightly corrupted PNGs [web:176]

class RobustHouseDataset(Dataset):
    def __init__(self, df, tabular_data, transform=None):
        self.df


In [8]:
# COMPLETE ROBUST DATASET + TRAINING
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True  # Allow corrupted PNGs [web:192][web:194]

# Robust dataset that skips BAD images
class RobustHouseDataset(Dataset):
    def __init__(self, df, tabular_data, transform=None):
        self.df = df
        self.tabular = tabular_data
        self.transform = transform
        self.valid_indices = []

        # Pre-filter valid images
        print("Checking images...")
        for i in tqdm(range(len(df))):
            try:
                img_path = df.iloc[i]['image_path']
                with Image.open(img_path) as img:
                    img.verify()  # Fast check
                self.valid_indices.append(i)
            except:
                pass  # Skip corrupted

        print(f"Valid images: {len(self.valid_indices)}/{len(df)}")

    def __len__(self):
        return len(self.valid_indices)

    def __getitem__(self, idx):
        real_idx = self.valid_indices[idx]
        img_path = self.df.iloc[real_idx]['image_path']

        # Safe image load
        try:
            image = Image.open(img_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except:
            # Fallback: black image
            image = torch.zeros(3, 224, 224)

        tabular = torch.FloatTensor(self.tabular[real_idx])
        price = torch.FloatTensor([self.df.iloc[real_idx]['price']])
        return image, tabular, price

# Recreate dataset
dataset = RobustHouseDataset(df_train_valid, X_tab_scaled, transform)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)  # Smaller batch


Checking images...


100%|██████████| 16209/16209 [1:29:20<00:00,  3.02it/s]

Valid images: 16208/16209


In [9]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using: {device}")

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()

model.train()
total_loss = 0
for batch_idx, (images, tabular, prices) in enumerate(tqdm(dataloader)):
    images, tabular, prices = images.to(device), tabular.to(device), prices.to(device)

    optimizer.zero_grad()
    pred_prices = model(images, tabular)
    loss = criterion(pred_prices.squeeze(), prices.squeeze())
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    if batch_idx % 20 == 0:
        print(f'Batch {batch_idx}, Loss: {loss.item():.2f}')

    if batch_idx >= 200:  # ~3200 samples
        break

print(f"Avg loss: {total_loss/len(dataloader):.2f}")

Using: cpu


  0%|          | 1/1013 [00:06<1:45:30,  6.26s/it]

Batch 0, Loss: 516811718656.00


  2%|▏         | 21/1013 [01:31<1:08:53,  4.17s/it]

Batch 20, Loss: 409796411392.00


...

Batch 160, Loss: 31241062400.00


 18%|█▊        | 181/1013 [12:41<1:00:46,  4.38s/it]

Batch 180, Loss: 41616293888.00


 20%|█▉        | 200/1013 [14:05<57:17,  4.23s/it]

Batch 200, Loss: 16115107840.00
Avg loss: 48715114748.71
